In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

'/home/ubuntu/varios/skforecast'

In [29]:
import numpy as np
import pandas as pd
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import grid_search_forecaster
from lightgbm import LGBMRegressor

In [37]:
y = pd.Series(np.random.normal(size=50))
exog_1 = pd.Series(['a']*25 + ['b']*25, name='exog_1', dtype='category')
exog_2 = pd.Series([np.nan]*25 + [2]*25, name='exog_2', dtype=float)
exog   = pd.concat((exog_1, exog_2), axis=1)

In [38]:
forecaster = ForecasterAutoreg(regressor=LGBMRegressor(), lags=5)

In [39]:
X_train, y_train = forecaster.create_train_X_y(
    y=y,
    exog=exog
)
print(X_train.dtypes)
X_train.head(3)

lag_1      float64
lag_2      float64
lag_3      float64
lag_4      float64
lag_5      float64
exog_1    category
exog_2     float64
dtype: object


/home/ubuntu/varios/skforecast/skforecast/utils/utils.py:229: UserWarning: `exog` has missing values. Most of machine learning models do not allow missing values.
  warnings.warn(


,lag_1,lag_2,lag_3,lag_4,lag_5,exog_1,exog_2
5,-0.174980,-0.019890,0.228193,-0.031656,0.481604,a,NaN
6,0.424086,-0.174980,-0.019890,0.228193,-0.031656,a,NaN
7,-0.672830,0.424086,-0.174980,-0.019890,0.228193,a,NaN


In [33]:
exog = exog_2

if isinstance(exog, pd.DataFrame):
    if not all([dtype in ['float', 'int', 'category'] for dtype in exog.dtypes]):
        raise TypeError(
            "Exog must contain only int, float or category types"
        )

if isinstance(exog, pd.Series):
    if not exog.dtypes in ['float', 'int', 'category']:
        raise TypeError(
            "Exog must contain only int, float or category types"
        )

check that categories only contain integers

TypeError: Exog must contain only int, float or category types

In [146]:
forecaster.fit(y=y, exog=exog)

In [147]:
exog_predict = exog.copy()
exog_predict.index = pd.RangeIndex(50, 100)
forecaster.predict(steps=3, exog=exog_predict)
exog.head(3)

ValueError: could not convert string to float: 'a'

In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector
from sklearn.preprocessing import OrdinalEncoder

categorical_cols = exog.select_dtypes(include="category").columns
ordinal_encoder = make_column_transformer(
    (
        OrdinalEncoder(dtype=int, handle_unknown="use_encoded_value", unknown_value=-1),
        make_column_selector(dtype_include="category"),
    ),
    remainder="passthrough",
    verbose_feature_names_out=False,
).set_output(transform="pandas")

exog_encoded = ordinal_encoder.fit_transform(exog)
exog_encoded[categorical_cols] = exog_encoded[categorical_cols].astype('category')

In [ ]:
X_train, y_train = forecaster.create_train_X_y(
    y=y,
    exog=exog_encoded
)
X_train.head(3)

,lag_1,lag_2,lag_3,lag_4,lag_5,exog_1,exog_2
5,0.662517,0.369443,-1.070102,0.072779,-0.292832,0,0
6,-1.156970,0.662517,0.369443,-1.070102,0.072779,0,0
7,-0.315181,-1.156970,0.662517,0.369443,-1.070102,0,0


In [ ]:
X_train.dtypes

lag_1      float64
lag_2      float64
lag_3      float64
lag_4      float64
lag_5      float64
exog_1    category
exog_2    category
dtype: object

In [ ]:
forecaster.fit(y=y, exog=exog_encoded)

In [ ]:
exog_predict = exog_encoded.copy()
exog_predict.index = pd.RangeIndex(50, 100)
forecaster.predict(steps=3, exog=exog_predict)

50   -0.593105
51    0.223661
52   -0.015733
Name: pred, dtype: float64

In [155]:
X=exog.copy()
model = LGBMRegressor()
model.fit(y=y.to_frame(), X=X)
model.predict(X)

array([-0.16857512, -0.16857512, -0.16857512, -0.16857512, -0.16857512,
       -0.16857512, -0.16857512, -0.16857512, -0.16857512, -0.16857512,
       -0.16857512, -0.16857512, -0.16857512, -0.16857512, -0.16857512,
       -0.16857512, -0.16857512, -0.16857512, -0.16857512, -0.16857512,
       -0.16857512, -0.16857512, -0.16857512, -0.16857512, -0.16857512,
       -0.19153426, -0.19153426, -0.19153426, -0.19153426, -0.19153426,
       -0.19153426, -0.19153426, -0.19153426, -0.19153426, -0.19153426,
       -0.19153426, -0.19153426, -0.19153426, -0.19153426, -0.19153426,
       -0.19153426, -0.19153426, -0.19153426, -0.19153426, -0.19153426,
       -0.19153426, -0.19153426, -0.19153426, -0.19153426, -0.19153426])

In [156]:
for col in X.columns:
    X[col] = X[col].cat.codes

In [157]:
model.predict(X.to_numpy())

array([-0.16857512, -0.16857512, -0.16857512, -0.16857512, -0.16857512,
       -0.16857512, -0.16857512, -0.16857512, -0.16857512, -0.16857512,
       -0.16857512, -0.16857512, -0.16857512, -0.16857512, -0.16857512,
       -0.16857512, -0.16857512, -0.16857512, -0.16857512, -0.16857512,
       -0.16857512, -0.16857512, -0.16857512, -0.16857512, -0.16857512,
       -0.19153426, -0.19153426, -0.19153426, -0.19153426, -0.19153426,
       -0.19153426, -0.19153426, -0.19153426, -0.19153426, -0.19153426,
       -0.19153426, -0.19153426, -0.19153426, -0.19153426, -0.19153426,
       -0.19153426, -0.19153426, -0.19153426, -0.19153426, -0.19153426,
       -0.19153426, -0.19153426, -0.19153426, -0.19153426, -0.19153426])